In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns



In [27]:
titles = []
years = []
imdb_ratings = []
text_muted1 = []
text_muted2 = []
text_muted3 = []
meta_scores = []
director = []
stars = []


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
for pages in np.arange(1,9900, 250):
    if pages == 1:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&ref_=adv_prv', headers=headers)
    else:
        response = get(
            'https://www.imdb.com/search/title/?title_type=feature&num_votes=7500,&languages=en&sort=user_rating,desc&count=250&start='+str(pages)+'&ref_=adv_nxt', headers=headers)
    
    sleep(randint(8, 15))

    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(
            requests, response.status_code))

    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select('.lister-item'):
        titles.append(item.h3.a.text)

        year = item.h3.find('span', class_='lister-item-year text-muted unbold')
        years.append(year.text if (year is not None) else None)

        text_muted = item.findAll('p', class_='text-muted')
        text_muted1.append(text_muted[0].text if (text_muted is not None) else None)
        text_muted2.append(text_muted[1].text if (text_muted is not None) else None)

        imdb_ratings.append(item.strong.text if (item.strong.text is not None) else None)

        meta_score = item.find('span', class_='metascore')
        meta_scores.append(meta_score.text if (meta_score is not None) else None)

        director.append([x.text for x in d] if (d := item.select('a[href*="_dr_"]')) else None)
        
        stars.append([x.text for x in s] if (s := item.select('a[href*="_st_"]')) else None)

        text_muted = item.find('p', class_='sort-num_votes-visible')
        text_muted3.append(text_muted.text if (text_muted is not None) else None)


In [28]:
df = pd.DataFrame({'titles': titles,
                   'year': years,
                   'imdb_ratings': imdb_ratings,
                   'text_muted1': text_muted1,
                   'text_muted2': text_muted2,
                   'text_muted3': text_muted3,
                   'meta_scores': meta_scores,
                   'director': director,
                   'stars': stars
                   })

df


AttributeError: Can only use .str accessor with string values!

In [ ]:
df.to_csv('imdb_scraping.csv')  